# 영상 받아서 이미지 한장씩 저장하기

In [1]:
import cv2
import os

filepath = './test111.mp4'
video = cv2.cv2.VideoCapture(filepath)

if not video.isOpened():
    print("Could not Open :",filepath)
    exit(0)
    
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 2004
width : 1920
height : 1080
fps : 30.0


In [2]:
new_path = './new_image/' # 이미지가 담길 폴더
import cv2
vidcap = cv2.VideoCapture(filepath)  # 영상이 있는 경로 
count = 0 

while(vidcap.isOpened()): 
    ret, image = vidcap.read() # 이미지 사이즈 960x540으로 변경 
    if ret:
        image = cv2.resize(image, (960, 540)) # 30프레임당 하나씩 이미지 추출 
        if(int(vidcap.get(1)) % 30 == 0): 
            print('Saved frame number : ' + str(int(vidcap.get(1)))) # 추출된 이미지가 저장되는 경로 
            cv2.imwrite(new_path+"frame%d.png" % count, image) #
            print('Saved frame%d.jpg' % count) 
            count += 1
    else:
        break
        
vidcap.release()

Saved frame number : 30
Saved frame0.jpg
Saved frame number : 60
Saved frame1.jpg
Saved frame number : 90
Saved frame2.jpg
Saved frame number : 120
Saved frame3.jpg
Saved frame number : 150
Saved frame4.jpg
Saved frame number : 180
Saved frame5.jpg
Saved frame number : 210
Saved frame6.jpg
Saved frame number : 240
Saved frame7.jpg
Saved frame number : 270
Saved frame8.jpg
Saved frame number : 300
Saved frame9.jpg
Saved frame number : 330
Saved frame10.jpg
Saved frame number : 360
Saved frame11.jpg
Saved frame number : 390
Saved frame12.jpg
Saved frame number : 420
Saved frame13.jpg
Saved frame number : 450
Saved frame14.jpg
Saved frame number : 480
Saved frame15.jpg
Saved frame number : 510
Saved frame16.jpg
Saved frame number : 540
Saved frame17.jpg
Saved frame number : 570
Saved frame18.jpg
Saved frame number : 600
Saved frame19.jpg
Saved frame number : 630
Saved frame20.jpg
Saved frame number : 660
Saved frame21.jpg
Saved frame number : 690
Saved frame22.jpg
Saved frame number : 720

# 연속된 이미지 불러와서 검출 -> 분류 -> 시각화

- 연속된 이미지 파일명 : 20200602_1.jpg (이런식으로 파일명을 변경함) 
- 여기서 검출된 신호등 20200602_1_1, 20200602_1_2 ... 이런식으로

In [3]:
import cv2
import matplotlib.pyplot as plt

## green yellow red 모델들

# 파일명 시작 숫자 설정
seq = 1 # 채번 변수 통일 20220308 by yjlim

# 추출 이미지 저장 위치 설정
# 이미지 한장으로 테스트 하는 거라서 디렉토리 분류 할 필요 없이 한 폴더에 다 담기
path = './vi_data2/'

first_detect = []

##### 1. green #####
def green(img, name, X, Y, W, H):
    global seq, path_g, path
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    name = name.split('.')
    
    # ROI 지정
    roi = B[Y:Y+H, X:X+W]
    roi2 = roi.copy()    

    ## 초록색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (43, 40, 120)
    up = (95, 255, 255)

    # 범위내의 픽셀들은 흰색, 나머지 검은색
    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    
    # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion2 = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)    
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)
    

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))
            
    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - 2 * length - 2 * interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,\
                        X + a[0] - 2 * length - 2 * interval_x:X + a[0] + length + interval_x]
                print(img + '///G' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y, a[1] + a[3] + interval_y, X + a[0] - 2 * length - 2 * interval_x, X + a[0] + length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 2. yellow #####
def yellow(img, name, X, Y, W, H):
    global seq, path_y, path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 노랑색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (10, 10, 110)
    up = (31, 255, 255)
    # low = (20, 20, 100)
    # up = (30, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(roi2_result, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(roi2_result, k)
    
    erosion3 = erosion1.copy()

    
    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3) # 너비 3등분
            interval_y = int(a[3] / 4) # 높이 4등분
            
            if a[1] - interval_y > 0 and X + a[0] - length - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - length - interval_x:X + a[0] + 2 * length + interval_x]
                
                print(img + '///Y' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y,X + a[0] - length - interval_x, X + a[0] + 2 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] + '_' + str(seq) + '.jpg', real3)
                seq = seq + 1
                
                
                
##### 3. red #####
def red(img, name, X, Y, W, H):
    global seq, path_r,path
    
    name = name.split('.')
    
    ## 파일 불러오기
    B = cv2.imread(img, cv2.IMREAD_COLOR)
    
    # ROI 지정
    roi = B[Y:Y+H,X:X+W]
    roi2 = roi.copy()

    ## 빨강색 부분 추출
    # hsv 영역 전환
    roi2_hsv = cv2.cvtColor(roi2, cv2.COLOR_BGR2HSV)

    # 색 범위 지정
    low = (0, 30,50)
    up = (10, 255, 255)

    roi2_mask = cv2.inRange(roi2_hsv, low, up)
    roi2_result = cv2.bitwise_and(roi2, roi2, mask = roi2_mask)
    
    # 색 영역 두번째 추가
    low3 = (150, 40, 50)
    up3 = (180, 255, 255)
    
    roi3_mask = cv2.inRange(roi2_hsv, low3, up3)
    roi3_result = cv2.bitwise_and(roi2, roi2, mask = roi3_mask)
    
    # 색 영역 1 + 색 영역 2  
    result_red = roi2_result + roi3_result

    ## 모폴 침식 사용
    # 구조화 요소 커널, 사각형 (2x2) 생성
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    
    # 열림 연산 적용 # yjlim 추가
    erosion1 = cv2.morphologyEx(result_red, cv2.MORPH_OPEN, k)
    
    # 침식 연산 적용
    erosion = cv2.erode(result_red, k)
    
    erosion3 = erosion1.copy()

    ## 색 영역 변환
    # FindContours support only 8uC1 and 32sC1 images,
    # HSV 이미지는 contour 기능을 쓸 수 없으므로 HSV->BGR->GRAY 로 전환하자
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_HSV2BGR)
    erosion3 = cv2.cvtColor(erosion3, cv2.COLOR_BGR2GRAY)

    ## contour 직사각형으로 출력
    position = []
    contours, hierarchy = cv2.findContours(erosion3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # cnt를 일일이 확인해서 좌표상으로 크게 차이 안나는 것들(잡음)은 추가안함(너무 작은 직사각형, 한쪽으로 길쭉한 직사각형 제외)
        if (w >= 7) and (h >= 7) and (abs(w - h) <= 6):
            cv2.rectangle(erosion3, (x, y), (x + w, y + h), (255, 0, 0), 1)
            position.append((x, y, w, h))

    ## 추출한 신호등만 별도의 이미지로 저장
    if position:
        for a in position:
            # 가로, 세로 높이 비교해서 큰 값으로 길이 설정 # yjlim
            if a[2] >= a[3]:
                length = a[2]
            else:
                length = a[3]
                
            interval_x = int(length / 3)
            interval_y = int(a[3] / 4)
            
            if a[1] - interval_y > 0 and X + a[0] - interval_x > 0:
                real3 = B[a[1] - interval_y:a[1] + a[3] + interval_y,
                        X + a[0] - interval_x:X + a[0] + 3 * length + interval_x]
                print(img + '///R' + str(seq) + '.jpg')
                first_detect.append([[a[1] - interval_y,a[1] + a[3] + interval_y, X + a[0] - interval_x, X + a[0] + 3 * length + interval_x], name[0] + '_' + str(seq)])
                cv2.imwrite(path + name[0] +'_'+ str(seq) + '.jpg', real3)
                seq = seq + 1

In [4]:
import sys
import glob
import cv2

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
# 연속된 이미지가 담긴 폴더
#img_files = glob.glob('./video/*.jpg')
img_files = glob.glob('.\\new_image\\*.png')
img_files.sort() # 순서대로 정렬

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
    
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
    if cv2.waitKey(1000) == 27:     
        break

    index += 1      
    
    # ROI 지정, x,y,w,h ,N 설정 완료
    x1 = 300
    y1 = 0
    w1 = 1400
    h1 = 500
    
    green(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    yellow(filepath, name, X = x1, Y = y1, W = w1, H = h1)
    red(filepath, name, X = x1, Y = y1, W = w1, H = h1)


.\new_image\frame0.png///G1.jpg
.\new_image\frame0.png///G2.jpg
.\new_image\frame0.png///G3.jpg
.\new_image\frame0.png///G4.jpg
.\new_image\frame0.png///G5.jpg
.\new_image\frame0.png///Y6.jpg
.\new_image\frame0.png///Y7.jpg
.\new_image\frame0.png///Y8.jpg
.\new_image\frame0.png///Y9.jpg
.\new_image\frame0.png///Y10.jpg
.\new_image\frame0.png///Y11.jpg
.\new_image\frame0.png///Y12.jpg
.\new_image\frame0.png///Y13.jpg
.\new_image\frame0.png///Y14.jpg
.\new_image\frame0.png///Y15.jpg
.\new_image\frame0.png///R16.jpg
.\new_image\frame0.png///R17.jpg
.\new_image\frame0.png///R18.jpg
.\new_image\frame0.png///R19.jpg
.\new_image\frame0.png///R20.jpg
.\new_image\frame0.png///R21.jpg
.\new_image\frame0.png///R22.jpg
.\new_image\frame0.png///R23.jpg
.\new_image\frame0.png///R24.jpg
.\new_image\frame0.png///R25.jpg
.\new_image\frame0.png///R26.jpg
.\new_image\frame0.png///R27.jpg
.\new_image\frame0.png///R28.jpg
.\new_image\frame0.png///R29.jpg
.\new_image\frame0.png///R30.jpg
.\new_image\frame0.

.\new_image\frame16.png///G253.jpg
.\new_image\frame16.png///Y254.jpg
.\new_image\frame16.png///Y255.jpg
.\new_image\frame16.png///Y256.jpg
.\new_image\frame16.png///Y257.jpg
.\new_image\frame16.png///Y258.jpg
.\new_image\frame16.png///Y259.jpg
.\new_image\frame16.png///Y260.jpg
.\new_image\frame16.png///Y261.jpg
.\new_image\frame16.png///Y262.jpg
.\new_image\frame16.png///Y263.jpg
.\new_image\frame16.png///Y264.jpg
.\new_image\frame16.png///R265.jpg
.\new_image\frame16.png///R266.jpg
.\new_image\frame16.png///R267.jpg
.\new_image\frame16.png///R268.jpg
.\new_image\frame16.png///R269.jpg
.\new_image\frame16.png///R270.jpg
.\new_image\frame16.png///R271.jpg
.\new_image\frame17.png///G272.jpg
.\new_image\frame17.png///G273.jpg
.\new_image\frame17.png///Y274.jpg
.\new_image\frame17.png///Y275.jpg
.\new_image\frame17.png///Y276.jpg
.\new_image\frame17.png///Y277.jpg
.\new_image\frame17.png///R278.jpg
.\new_image\frame17.png///R279.jpg
.\new_image\frame17.png///R280.jpg
.\new_image\frame17.

.\new_image\frame24.png///Y495.jpg
.\new_image\frame24.png///Y496.jpg
.\new_image\frame24.png///Y497.jpg
.\new_image\frame24.png///Y498.jpg
.\new_image\frame24.png///Y499.jpg
.\new_image\frame24.png///Y500.jpg
.\new_image\frame24.png///Y501.jpg
.\new_image\frame24.png///Y502.jpg
.\new_image\frame24.png///Y503.jpg
.\new_image\frame24.png///Y504.jpg
.\new_image\frame24.png///Y505.jpg
.\new_image\frame24.png///Y506.jpg
.\new_image\frame24.png///Y507.jpg
.\new_image\frame24.png///Y508.jpg
.\new_image\frame24.png///Y509.jpg
.\new_image\frame24.png///Y510.jpg
.\new_image\frame24.png///Y511.jpg
.\new_image\frame24.png///Y512.jpg
.\new_image\frame24.png///Y513.jpg
.\new_image\frame24.png///Y514.jpg
.\new_image\frame24.png///Y515.jpg
.\new_image\frame24.png///Y516.jpg
.\new_image\frame24.png///Y517.jpg
.\new_image\frame24.png///Y518.jpg
.\new_image\frame24.png///Y519.jpg
.\new_image\frame24.png///Y520.jpg
.\new_image\frame24.png///Y521.jpg
.\new_image\frame24.png///R522.jpg
.\new_image\frame24.

.\new_image\frame3.png///Y757.jpg
.\new_image\frame3.png///Y758.jpg
.\new_image\frame3.png///Y759.jpg
.\new_image\frame3.png///Y760.jpg
.\new_image\frame3.png///Y761.jpg
.\new_image\frame3.png///Y762.jpg
.\new_image\frame3.png///Y763.jpg
.\new_image\frame3.png///Y764.jpg
.\new_image\frame3.png///Y765.jpg
.\new_image\frame3.png///Y766.jpg
.\new_image\frame3.png///Y767.jpg
.\new_image\frame3.png///Y768.jpg
.\new_image\frame3.png///Y769.jpg
.\new_image\frame3.png///Y770.jpg
.\new_image\frame3.png///Y771.jpg
.\new_image\frame3.png///Y772.jpg
.\new_image\frame3.png///Y773.jpg
.\new_image\frame3.png///Y774.jpg
.\new_image\frame3.png///Y775.jpg
.\new_image\frame3.png///R776.jpg
.\new_image\frame3.png///R777.jpg
.\new_image\frame3.png///R778.jpg
.\new_image\frame3.png///R779.jpg
.\new_image\frame3.png///R780.jpg
.\new_image\frame3.png///R781.jpg
.\new_image\frame3.png///R782.jpg
.\new_image\frame3.png///R783.jpg
.\new_image\frame3.png///R784.jpg
.\new_image\frame3.png///R785.jpg
.\new_image\fr

.\new_image\frame37.png///Y1018.jpg
.\new_image\frame37.png///Y1019.jpg
.\new_image\frame37.png///Y1020.jpg
.\new_image\frame37.png///Y1021.jpg
.\new_image\frame37.png///Y1022.jpg
.\new_image\frame37.png///Y1023.jpg
.\new_image\frame37.png///Y1024.jpg
.\new_image\frame37.png///Y1025.jpg
.\new_image\frame37.png///Y1026.jpg
.\new_image\frame37.png///Y1027.jpg
.\new_image\frame37.png///Y1028.jpg
.\new_image\frame37.png///Y1029.jpg
.\new_image\frame37.png///Y1030.jpg
.\new_image\frame37.png///Y1031.jpg
.\new_image\frame37.png///Y1032.jpg
.\new_image\frame37.png///R1033.jpg
.\new_image\frame37.png///R1034.jpg
.\new_image\frame37.png///R1035.jpg
.\new_image\frame37.png///R1036.jpg
.\new_image\frame37.png///R1037.jpg
.\new_image\frame37.png///R1038.jpg
.\new_image\frame37.png///R1039.jpg
.\new_image\frame37.png///R1040.jpg
.\new_image\frame37.png///R1041.jpg
.\new_image\frame37.png///R1042.jpg
.\new_image\frame37.png///R1043.jpg
.\new_image\frame37.png///R1044.jpg
.\new_image\frame37.png///R1

.\new_image\frame43.png///Y1278.jpg
.\new_image\frame43.png///Y1279.jpg
.\new_image\frame43.png///Y1280.jpg
.\new_image\frame43.png///Y1281.jpg
.\new_image\frame43.png///Y1282.jpg
.\new_image\frame43.png///Y1283.jpg
.\new_image\frame43.png///Y1284.jpg
.\new_image\frame43.png///Y1285.jpg
.\new_image\frame43.png///Y1286.jpg
.\new_image\frame43.png///Y1287.jpg
.\new_image\frame43.png///Y1288.jpg
.\new_image\frame43.png///Y1289.jpg
.\new_image\frame43.png///Y1290.jpg
.\new_image\frame43.png///Y1291.jpg
.\new_image\frame43.png///Y1292.jpg
.\new_image\frame43.png///Y1293.jpg
.\new_image\frame43.png///Y1294.jpg
.\new_image\frame43.png///Y1295.jpg
.\new_image\frame43.png///Y1296.jpg
.\new_image\frame43.png///Y1297.jpg
.\new_image\frame43.png///R1298.jpg
.\new_image\frame43.png///R1299.jpg
.\new_image\frame43.png///R1300.jpg
.\new_image\frame43.png///R1301.jpg
.\new_image\frame43.png///R1302.jpg
.\new_image\frame43.png///R1303.jpg
.\new_image\frame43.png///R1304.jpg
.\new_image\frame43.png///R1

.\new_image\frame5.png///Y1513.jpg
.\new_image\frame5.png///Y1514.jpg
.\new_image\frame5.png///Y1515.jpg
.\new_image\frame5.png///Y1516.jpg
.\new_image\frame5.png///Y1517.jpg
.\new_image\frame5.png///Y1518.jpg
.\new_image\frame5.png///Y1519.jpg
.\new_image\frame5.png///Y1520.jpg
.\new_image\frame5.png///Y1521.jpg
.\new_image\frame5.png///Y1522.jpg
.\new_image\frame5.png///Y1523.jpg
.\new_image\frame5.png///Y1524.jpg
.\new_image\frame5.png///Y1525.jpg
.\new_image\frame5.png///Y1526.jpg
.\new_image\frame5.png///Y1527.jpg
.\new_image\frame5.png///R1528.jpg
.\new_image\frame5.png///R1529.jpg
.\new_image\frame5.png///R1530.jpg
.\new_image\frame5.png///R1531.jpg
.\new_image\frame5.png///R1532.jpg
.\new_image\frame5.png///R1533.jpg
.\new_image\frame5.png///R1534.jpg
.\new_image\frame5.png///R1535.jpg
.\new_image\frame5.png///R1536.jpg
.\new_image\frame5.png///R1537.jpg
.\new_image\frame5.png///R1538.jpg
.\new_image\frame5.png///R1539.jpg
.\new_image\frame5.png///R1540.jpg
.\new_image\frame5.p

.\new_image\frame56.png///G1761.jpg
.\new_image\frame56.png///Y1762.jpg
.\new_image\frame56.png///Y1763.jpg
.\new_image\frame56.png///Y1764.jpg
.\new_image\frame56.png///Y1765.jpg
.\new_image\frame56.png///Y1766.jpg
.\new_image\frame56.png///Y1767.jpg
.\new_image\frame56.png///Y1768.jpg
.\new_image\frame56.png///Y1769.jpg
.\new_image\frame56.png///Y1770.jpg
.\new_image\frame56.png///Y1771.jpg
.\new_image\frame56.png///Y1772.jpg
.\new_image\frame56.png///Y1773.jpg
.\new_image\frame56.png///Y1774.jpg
.\new_image\frame56.png///Y1775.jpg
.\new_image\frame56.png///Y1776.jpg
.\new_image\frame56.png///Y1777.jpg
.\new_image\frame56.png///Y1778.jpg
.\new_image\frame56.png///Y1779.jpg
.\new_image\frame56.png///Y1780.jpg
.\new_image\frame56.png///Y1781.jpg
.\new_image\frame56.png///Y1782.jpg
.\new_image\frame56.png///Y1783.jpg
.\new_image\frame56.png///R1784.jpg
.\new_image\frame56.png///R1785.jpg
.\new_image\frame56.png///R1786.jpg
.\new_image\frame56.png///R1787.jpg
.\new_image\frame56.png///R1

.\new_image\frame63.png///Y2020.jpg
.\new_image\frame63.png///Y2021.jpg
.\new_image\frame63.png///Y2022.jpg
.\new_image\frame63.png///Y2023.jpg
.\new_image\frame63.png///Y2024.jpg
.\new_image\frame63.png///Y2025.jpg
.\new_image\frame63.png///Y2026.jpg
.\new_image\frame63.png///Y2027.jpg
.\new_image\frame63.png///Y2028.jpg
.\new_image\frame63.png///Y2029.jpg
.\new_image\frame63.png///Y2030.jpg
.\new_image\frame63.png///Y2031.jpg
.\new_image\frame63.png///Y2032.jpg
.\new_image\frame63.png///Y2033.jpg
.\new_image\frame63.png///Y2034.jpg
.\new_image\frame63.png///Y2035.jpg
.\new_image\frame63.png///Y2036.jpg
.\new_image\frame63.png///Y2037.jpg
.\new_image\frame63.png///Y2038.jpg
.\new_image\frame63.png///Y2039.jpg
.\new_image\frame63.png///Y2040.jpg
.\new_image\frame63.png///Y2041.jpg
.\new_image\frame63.png///Y2042.jpg
.\new_image\frame63.png///Y2043.jpg
.\new_image\frame63.png///Y2044.jpg
.\new_image\frame63.png///Y2045.jpg
.\new_image\frame63.png///R2046.jpg
.\new_image\frame63.png///R2

In [5]:
import os
import cv2
import numpy as np
import random
from PIL import Image

from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf

### 신호등 인식 모델

In [6]:
model_detection = tf.keras.models.load_model('traffic_light_detection_v7.h5')

#### 이미지 resize 해서 해당 폴더에 다시 저장

In [7]:
path_detection = 'vi_data2/' # 색 영역으로 검출한 모든 신호등 후보 영역 이미지

file_list_d = os.listdir(path_detection)

if ".DS_Store" in file_list_d:
    file_list_d.remove(".DS_Store")

data_detection = []

for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img_d = cv2.imread(path_detection + A)
    res_d = cv2.resize(img_d, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(path_detection + A, res_d)
    
for A in file_list_d:
    if A == ".DS_Store":
        continue
        
    img3 = Image.open(path_detection + A)
    data_detection.append(np.array(img3))
    img3.close()

In [8]:
data_detection = np.asarray(data_detection) / 255.0

In [9]:
prediction = model_detection.predict(data_detection)

In [10]:
light_order = {0 : "negative", 1 : "positive"}

light_detect_img = [] # 파일명, 예측 결과(negative, positive)
data_detection2 = [] # 신호등 이미지

for i, x in enumerate(prediction):
    pred_detect = light_order[np.argmax(x)] # 확률이 더 큰 값
    
    # 신호등으로 90퍼센트 이상 인식된 이미지만 추출
    if pred_detect == "positive":
        if x[1] >= 0.90:
            print(str(i) + "번째 신호등: " + file_list_d[i] + " ==> " + light_order[np.argmax(x)] + " ::: " + str(x[1]))
            light_detect_img.append([file_list_d[i], light_order[np.argmax(x)]])
            
            imgPath = path_detection + file_list_d[i]
            
            img_detection = Image.open(imgPath)
            data_detection2.append(np.array(img_detection)) # 신호등으로 인식된 이미지

0번째 신호등: frame0_1.jpg ==> positive ::: 1.0
11번째 신호등: frame0_2.jpg ==> positive ::: 1.0
22번째 신호등: frame0_3.jpg ==> positive ::: 1.0
28번째 신호등: frame0_4.jpg ==> positive ::: 1.0
29번째 신호등: frame0_5.jpg ==> positive ::: 1.0
34번째 신호등: frame10_100.jpg ==> positive ::: 0.9963063
67번째 신호등: frame11_107.jpg ==> positive ::: 0.9999912
86번째 신호등: frame11_126.jpg ==> positive ::: 0.93048304
121번째 신호등: frame12_161.jpg ==> positive ::: 0.9997054
156번째 신호등: frame13_196.jpg ==> positive ::: 0.9999989
157번째 신호등: frame13_197.jpg ==> positive ::: 0.99999547
158번째 신호등: frame13_198.jpg ==> positive ::: 0.96483517
174번째 신호등: frame13_214.jpg ==> positive ::: 1.0
175번째 신호등: frame13_215.jpg ==> positive ::: 0.9999566
231번째 신호등: frame16_271.jpg ==> positive ::: 1.0
249번째 신호등: frame17_289.jpg ==> positive ::: 1.0
308번째 신호등: frame1_35.jpg ==> positive ::: 1.0
309번째 신호등: frame1_36.jpg ==> positive ::: 1.0
310번째 신호등: frame1_37.jpg ==> positive ::: 1.0
712번째 신호등: frame2_348.jpg ==> positive ::: 1.0
943번째 신호등: frame35_9

In [11]:
len(data_detection2)

62

In [12]:
light_detect_img

[['frame0_1.jpg', 'positive'],
 ['frame0_2.jpg', 'positive'],
 ['frame0_3.jpg', 'positive'],
 ['frame0_4.jpg', 'positive'],
 ['frame0_5.jpg', 'positive'],
 ['frame10_100.jpg', 'positive'],
 ['frame11_107.jpg', 'positive'],
 ['frame11_126.jpg', 'positive'],
 ['frame12_161.jpg', 'positive'],
 ['frame13_196.jpg', 'positive'],
 ['frame13_197.jpg', 'positive'],
 ['frame13_198.jpg', 'positive'],
 ['frame13_214.jpg', 'positive'],
 ['frame13_215.jpg', 'positive'],
 ['frame16_271.jpg', 'positive'],
 ['frame17_289.jpg', 'positive'],
 ['frame1_35.jpg', 'positive'],
 ['frame1_36.jpg', 'positive'],
 ['frame1_37.jpg', 'positive'],
 ['frame2_348.jpg', 'positive'],
 ['frame35_978.jpg', 'positive'],
 ['frame35_979.jpg', 'positive'],
 ['frame37_1041.jpg', 'positive'],
 ['frame37_1042.jpg', 'positive'],
 ['frame38_1085.jpg', 'positive'],
 ['frame38_1086.jpg', 'positive'],
 ['frame39_1120.jpg', 'positive'],
 ['frame39_1121.jpg', 'positive'],
 ['frame39_1122.jpg', 'positive'],
 ['frame40_1200.jpg', 'positi

### 신호등 색 분류

In [13]:
model = tf.keras.models.load_model('traffic_light_0315_2.h5')

In [14]:
# # 검출된 신호등 이미지를 입력 값으로 만들기
# Path2 = 'vi_data2/'
# file_list = os.listdir(Path2)
# file_list.sort()
# if ".DS_Store" in file_list:
#     file_list.remove(".DS_Store")

# data2 = []

# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img2 = cv2.imread(Path2+A)
#         res2 = cv2.resize(img2, dsize = (45, 15), interpolation = cv2.INTER_CUBIC)
#         cv2.imwrite(Path2+A,res2)
#     except Exception as e:
#         print(str(e))
    
# for A in file_list:
#     if A == ".DS_Store":
#         continue
#     try:
#         img3 = Image.open(Path2 + A)
#         data2.append(np.array(img3))
#         img3.close()
#     except Exception as e:
#         print(str(e))

In [15]:
# data2

In [16]:
# data2 = np.asarray(data2) / 255.0
data_detection2 = np.asarray(data_detection2) / 255.0
prediction = model.predict(data_detection2)

In [17]:
prediction

array([[9.99922156e-01, 4.15951545e-05, 3.62075225e-05],
       [9.99876618e-01, 6.27733461e-05, 6.05416753e-05],
       [9.99860883e-01, 6.92605827e-05, 6.98271033e-05],
       [9.99972582e-01, 1.46350221e-05, 1.27976327e-05],
       [9.99763668e-01, 1.31444773e-04, 1.04862687e-04],
       [9.56101634e-04, 9.98711824e-01, 3.32037365e-04],
       [9.28312123e-01, 3.25530097e-02, 3.91349159e-02],
       [6.54931292e-02, 1.58924147e-01, 7.75582731e-01],
       [6.19040384e-06, 2.87399075e-06, 9.99990940e-01],
       [9.79328775e-07, 3.40515101e-07, 9.99998689e-01],
       [9.06332502e-07, 2.98222460e-07, 9.99998808e-01],
       [2.22807030e-05, 1.15922940e-05, 9.99966145e-01],
       [5.45430190e-09, 1.00000000e+00, 2.04538178e-10],
       [1.27730775e-04, 9.99847293e-01, 2.49618042e-05],
       [6.04281411e-07, 9.99999404e-01, 5.28373825e-08],
       [9.10530798e-06, 9.99989748e-01, 1.15386149e-06],
       [9.98315215e-01, 5.74424397e-04, 1.11040275e-03],
       [9.99961615e-01, 2.11977

In [18]:
len(first_detect) # 처음 색영역으로 신호등 후보 영역 검출한 개수

2248

In [19]:
light_order = {0 : "green", 1 : "red", 2 : "yellow", 3 : "unknown"}

# 신호등 파일명과 색을 한 리스트에 담기 
detect_img = [] 

for i, x in enumerate(prediction):
    count = 0
    for y in x:
        if y > 0.90:    # 90퍼센트 이상이면 인식
            count += 1
            
    if count > 0:
        print(str(i) + "번째 신호등: " + light_detect_img[i][0] + " : " + light_order[np.argmax(x)])
        detect_img.append([light_detect_img[i][0], light_order[np.argmax(x)]])
    else:
        print(str(i)+"번째 신호등: " + light_detect_img[i][0] + " : " + light_order[3])

0번째 신호등: frame0_1.jpg : green
1번째 신호등: frame0_2.jpg : green
2번째 신호등: frame0_3.jpg : green
3번째 신호등: frame0_4.jpg : green
4번째 신호등: frame0_5.jpg : green
5번째 신호등: frame10_100.jpg : red
6번째 신호등: frame11_107.jpg : green
7번째 신호등: frame11_126.jpg : unknown
8번째 신호등: frame12_161.jpg : yellow
9번째 신호등: frame13_196.jpg : yellow
10번째 신호등: frame13_197.jpg : yellow
11번째 신호등: frame13_198.jpg : yellow
12번째 신호등: frame13_214.jpg : red
13번째 신호등: frame13_215.jpg : red
14번째 신호등: frame16_271.jpg : red
15번째 신호등: frame17_289.jpg : red
16번째 신호등: frame1_35.jpg : green
17번째 신호등: frame1_36.jpg : green
18번째 신호등: frame1_37.jpg : green
19번째 신호등: frame2_348.jpg : green
20번째 신호등: frame35_978.jpg : red
21번째 신호등: frame35_979.jpg : red
22번째 신호등: frame37_1041.jpg : red
23번째 신호등: frame37_1042.jpg : red
24번째 신호등: frame38_1085.jpg : red
25번째 신호등: frame38_1086.jpg : red
26번째 신호등: frame39_1120.jpg : red
27번째 신호등: frame39_1121.jpg : red
28번째 신호등: frame39_1122.jpg : unknown
29번째 신호등: frame40_1200.jpg : red
30번째 신호등: frame40_1201.j

In [20]:
detect_img

[['frame0_1.jpg', 'green'],
 ['frame0_2.jpg', 'green'],
 ['frame0_3.jpg', 'green'],
 ['frame0_4.jpg', 'green'],
 ['frame0_5.jpg', 'green'],
 ['frame10_100.jpg', 'red'],
 ['frame11_107.jpg', 'green'],
 ['frame12_161.jpg', 'yellow'],
 ['frame13_196.jpg', 'yellow'],
 ['frame13_197.jpg', 'yellow'],
 ['frame13_198.jpg', 'yellow'],
 ['frame13_214.jpg', 'red'],
 ['frame13_215.jpg', 'red'],
 ['frame16_271.jpg', 'red'],
 ['frame17_289.jpg', 'red'],
 ['frame1_35.jpg', 'green'],
 ['frame1_36.jpg', 'green'],
 ['frame1_37.jpg', 'green'],
 ['frame2_348.jpg', 'green'],
 ['frame35_978.jpg', 'red'],
 ['frame35_979.jpg', 'red'],
 ['frame37_1041.jpg', 'red'],
 ['frame37_1042.jpg', 'red'],
 ['frame38_1085.jpg', 'red'],
 ['frame38_1086.jpg', 'red'],
 ['frame39_1120.jpg', 'red'],
 ['frame39_1121.jpg', 'red'],
 ['frame40_1200.jpg', 'red'],
 ['frame40_1201.jpg', 'red'],
 ['frame40_1202.jpg', 'red'],
 ['frame41_1230.jpg', 'red'],
 ['frame41_1231.jpg', 'red'],
 ['frame42_1257.jpg', 'yellow'],
 ['frame42_1271.jp

In [21]:
print(len(file_list_d),len(detect_img)) # 모델에 적용한 이미지, 이미지 검출 코드에서 나온거

2248 60


In [22]:
xy = []
for i in range(0, len(detect_img)):
    for j in range(0, len(first_detect)):

        if first_detect[j][1] == detect_img[i][0].split(".")[0]:
            xy.append([first_detect[j][1], first_detect[j][0], detect_img[i][1]])
    #print(detect_img[i][0].split(".")[0])

In [23]:
xy

[['frame0_1', [294, 303, 344, 374], 'green'],
 ['frame0_2', [290, 303, 401, 437], 'green'],
 ['frame0_3', [286, 299, 486, 522], 'green'],
 ['frame0_4', [17, 47, 332, 410], 'green'],
 ['frame0_5', [3, 30, 635, 710], 'green'],
 ['frame10_100', [322, 337, 934, 973], 'red'],
 ['frame11_107', [296, 308, 470, 500], 'green'],
 ['frame12_161', [251, 260, 610, 638], 'yellow'],
 ['frame13_196', [140, 160, 459, 517], 'yellow'],
 ['frame13_197', [132, 151, 614, 664], 'yellow'],
 ['frame13_198', [122, 143, 763, 818], 'yellow'],
 ['frame13_214', [129, 148, 630, 685], 'red'],
 ['frame13_215', [120, 145, 784, 845], 'red'],
 ['frame16_271', [249, 267, 568, 618], 'red'],
 ['frame17_289', [27, 71, 684, 800], 'red'],
 ['frame1_35', [246, 259, 276, 312], 'green'],
 ['frame1_36', [242, 256, 362, 404], 'green'],
 ['frame1_37', [233, 253, 484, 538], 'green'],
 ['frame2_348', [53, 83, 508, 586], 'green'],
 ['frame35_978', [324, 333, 397, 425], 'red'],
 ['frame35_979', [322, 331, 560, 588], 'red'],
 ['frame37_1

In [24]:
array = []
for i in range(0, len(xy)):
    coor = xy[i][1]
    color = xy[i][2]
    for j in range(i+1, len(xy)):
        if xy[i][0] == xy[j][0]:
            coor = xy[j][1]
            color = xy[j][2]
    if [xy[i][0], coor, color] not in array:
        array.append([xy[i][0], coor, color])
    #print(xy[i])

In [25]:
array

[['frame0_1', [294, 303, 344, 374], 'green'],
 ['frame0_2', [290, 303, 401, 437], 'green'],
 ['frame0_3', [286, 299, 486, 522], 'green'],
 ['frame0_4', [17, 47, 332, 410], 'green'],
 ['frame0_5', [3, 30, 635, 710], 'green'],
 ['frame10_100', [322, 337, 934, 973], 'red'],
 ['frame11_107', [296, 308, 470, 500], 'green'],
 ['frame12_161', [251, 260, 610, 638], 'yellow'],
 ['frame13_196', [140, 160, 459, 517], 'yellow'],
 ['frame13_197', [132, 151, 614, 664], 'yellow'],
 ['frame13_198', [122, 143, 763, 818], 'yellow'],
 ['frame13_214', [129, 148, 630, 685], 'red'],
 ['frame13_215', [120, 145, 784, 845], 'red'],
 ['frame16_271', [249, 267, 568, 618], 'red'],
 ['frame17_289', [27, 71, 684, 800], 'red'],
 ['frame1_35', [246, 259, 276, 312], 'green'],
 ['frame1_36', [242, 256, 362, 404], 'green'],
 ['frame1_37', [233, 253, 484, 538], 'green'],
 ['frame2_348', [53, 83, 508, 586], 'green'],
 ['frame35_978', [324, 333, 397, 425], 'red'],
 ['frame35_979', [322, 331, 560, 588], 'red'],
 ['frame37_1

# array에 담긴 파일명으로 원천 이미지에 시각화

- path 경로에서 원천 이미지 찾아서 그 이미지에다가 rectangle 그리는 거여서 덮어씌워짐  
- 덮어씌워지니까 원천 이미지는 꼭 백업해둘것

In [26]:
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0] + '.png'
    print(path,imgname)

frame0_1 [294, 303, 344, 374] green
./new_image/ frame0.png
frame0_2 [290, 303, 401, 437] green
./new_image/ frame0.png
frame0_3 [286, 299, 486, 522] green
./new_image/ frame0.png
frame0_4 [17, 47, 332, 410] green
./new_image/ frame0.png
frame0_5 [3, 30, 635, 710] green
./new_image/ frame0.png
frame10_100 [322, 337, 934, 973] red
./new_image/ frame10.png
frame11_107 [296, 308, 470, 500] green
./new_image/ frame11.png
frame12_161 [251, 260, 610, 638] yellow
./new_image/ frame12.png
frame13_196 [140, 160, 459, 517] yellow
./new_image/ frame13.png
frame13_197 [132, 151, 614, 664] yellow
./new_image/ frame13.png
frame13_198 [122, 143, 763, 818] yellow
./new_image/ frame13.png
frame13_214 [129, 148, 630, 685] red
./new_image/ frame13.png
frame13_215 [120, 145, 784, 845] red
./new_image/ frame13.png
frame16_271 [249, 267, 568, 618] red
./new_image/ frame16.png
frame17_289 [27, 71, 684, 800] red
./new_image/ frame17.png
frame1_35 [246, 259, 276, 312] green
./new_image/ frame1.png
frame1_36 [2

In [27]:
# 원천이미지 있는 디렉토리 경로
# 해당 디렉토리 백업해두기
path = './new_image/'
font = cv2.FONT_HERSHEY_SIMPLEX 

for a,b,c in array:
    print(a,b,c)
    imgname = a.split('_')[0]+'.png'
    img = cv2.imread(path+imgname)
    y,h,x,w = b
    w = w - x
    h = h - y
    text = c 
    org = (x, y - 8)
    
    if x > 0 and y > 0 and w > 0 and h > 0:
        cv2.rectangle(img, (x, y, w, h), (139, 255, 224), 2)
        cv2.putText(img, text, org, font, 0.7, (139, 255, 224), 2)
        
    # 기존 원천 이미지에 덮어쓰기 되는거니까 주의!!!
    cv2.imwrite(path + imgname, img)


    #print(a,b,c)

frame0_1 [294, 303, 344, 374] green
frame0_2 [290, 303, 401, 437] green
frame0_3 [286, 299, 486, 522] green
frame0_4 [17, 47, 332, 410] green
frame0_5 [3, 30, 635, 710] green
frame10_100 [322, 337, 934, 973] red
frame11_107 [296, 308, 470, 500] green
frame12_161 [251, 260, 610, 638] yellow
frame13_196 [140, 160, 459, 517] yellow
frame13_197 [132, 151, 614, 664] yellow
frame13_198 [122, 143, 763, 818] yellow
frame13_214 [129, 148, 630, 685] red
frame13_215 [120, 145, 784, 845] red
frame16_271 [249, 267, 568, 618] red
frame17_289 [27, 71, 684, 800] red
frame1_35 [246, 259, 276, 312] green
frame1_36 [242, 256, 362, 404] green
frame1_37 [233, 253, 484, 538] green
frame2_348 [53, 83, 508, 586] green
frame35_978 [324, 333, 397, 425] red
frame35_979 [322, 331, 560, 588] red
frame37_1041 [312, 324, 514, 547] red
frame37_1042 [310, 324, 354, 390] red
frame38_1085 [302, 314, 514, 550] red
frame38_1086 [300, 312, 336, 369] red
frame39_1120 [296, 310, 514, 550] red
frame39_1121 [294, 309, 581, 625

# 원천 이미지 연속 출력 (검출된 내용 반영)

#### 숫자 정렬용 라이브러리 설치

In [28]:
# !pip install natsort

In [29]:
import sys
import glob
import cv2

import natsort # 숫자 정렬용 라이브러리

# images에 있는 모든 jpg 파일을 img_files 리스트에 추가
img_files = glob.glob('.\\new_image\\*.png')
# img_files.sort()

# 문자열로 인식해서 [1, 10, ..., 2] 이런식으로 나오는 오류
# natsort 사용하여 [1, 2, ..., 10] 순으로 나올 수 있도록 정렬
img_files = natsort.natsorted(img_files)

# 이미지 없을때 예외처리
if not img_files:
    print("jpg 이미지가 없다..")
    sys.exit()


# 슬라이드 쇼 반복을 위한 반복문
count = len(img_files)
index = 0

while index < count:
    img = cv2.imread(img_files[index])
    name = img_files[index].split('\\')[2]
    filepath = img_files[index]
	# 예외처리
    if img is None:     
        print("이미지를 불러오는데 실패했습니다.")
        break
        
	# ESC가 입력되면 break
    cv2.imshow('video_test', img)
    if cv2.waitKey(1000) == 27:     
        break

	# index가 이미지 리스트보다 커지거나 같아지면 다시 0으로
    index += 1      
    
    #if index >= count :
        #index = 0 

cv2.waitKey(0)    
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# 동영상으로 저장

In [31]:
import numpy as np
import glob
 
img_array = []

filename_list = glob.glob('.\\new_image\\*.png')
filename_list = natsort.natsorted(filename_list)

for filename in filename_list:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('video_test.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()